In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install ultralytics

In [5]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
from IPython.display import Video
from ultralytics import YOLO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [6]:
import os

# Define the directory paths
base_dir = "yolov8_tracking"
examples_dir = os.path.join(base_dir, "examples")
weights_dir = os.path.join(examples_dir, "weights")

# Create the directories (with parents if needed)
os.makedirs(weights_dir, exist_ok=True)

print(f"Directories created: {base_dir}, {examples_dir}, {weights_dir}")

Directories created: yolov8_tracking, yolov8_tracking\examples, yolov8_tracking\examples\weights


In [7]:
import os

dirs = [
    "yolov8_tracking",
    "yolov8_tracking/examples",
    "yolov8_tracking/examples/weights"
]

for directory in dirs:
    os.makedirs(directory, exist_ok=True)

print("✅ Directories created successfully!")

✅ Directories created successfully!


In [8]:
import os
import shutil

# Define paths for Kaggle
kaggle_working_dir = "/kaggle/working/yolov8_tracking"
kaggle_input_video = "/kaggle/input/road-traffic-video-monitoring/dhaka_traffic.mp4"
kaggle_output_video = "/kaggle/working/yolov8_tracking/sample.mp4"

# Define paths for Windows
windows_input_video = r"C:\Users\madire sathwik\Downloads"
windows_working_dir = r"C:\Users\madire sathwik\yolov8_tracking"

# Check if running on Kaggle or Windows
if os.path.exists("/kaggle"):
    # Running on Kaggle
    os.makedirs(kaggle_working_dir, exist_ok=True)  # Create dir if it doesn't exist
    shutil.copy(kaggle_input_video, kaggle_output_video)
    input_video_path = kaggle_output_video
else:
    # Running on Windows
    os.makedirs(windows_working_dir, exist_ok=True)  # Create dir if it doesn't exist
    input_video_path = windows_input_video  # Use local path

print(f"Video path set to: {input_video_path}")


Video path set to: C:\Users\madire sathwik\Downloads


In [9]:
from IPython.display import Video

input_video_path = r"C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4"

Video(input_video_path, width=640, height=400, embed=True)

In [10]:
source="C:\\Users\\madire sathwik\\OneDrive\\Desktop\\vehicles video.mp4"


In [11]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [30]:
!yolo mode=track tracker="bytetrack.yaml" model=yolov8n.pt conf=0.1 source="C:\\Users\\madire sathwik\\OneDrive\\Desktop\\vehicles video.mp4" save=True save_txt=True


Ultralytics 8.3.65 ðŸš€ Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

video 1/1 (frame 1/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 2 cars, 4 trucks, 83.2ms
video 1/1 (frame 2/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 2 cars, 4 trucks, 57.2ms
video 1/1 (frame 3/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 3 cars, 3 trucks, 59.6ms
video 1/1 (frame 4/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 3 cars, 3 trucks, 65.5ms
video 1/1 (frame 5/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 2 cars, 3 trucks, 54.3ms
video 1/1 (frame 6/343) C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4: 384x640 1 person, 2 cars, 3 trucks, 58.6ms
video 1/1 (frame 7/343) C:\Users\madire sathwi

In [13]:
pip install ultralytics opencv-python


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install ultralytics deep_sort_realtime opencv-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize YOLOv8 model (download it if not present)
model = YOLO("yolov8n.pt")  # Use 'yolov8n.pt' or better model like 'yolov8m.pt'

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

# Load video file (replace with your video path)
video_path =r"C:\Users\madire sathwik\OneDrive\Desktop\vehicles video.mp4"
cap = cv2.VideoCapture(video_path)

# Select the target car ID
target_id = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video.")
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)[0]

    detections = []

    # Iterate through detected objects
    for box in results.boxes:
        class_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Filter: we only want cars (COCO class_id = 2)
        if class_id == 2 and conf > 0.3:  
            bbox = [x1, y1, x2 - x1, y2 - y1]  # Format: x, y, w, h
            detections.append((bbox, conf, class_id))

    # Update tracker with detections
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = track.to_ltrb()

        # Select the first detected car and assign target_id
        if target_id is None:
            target_id = track_id
            print(f"Tracking car with ID: {target_id}")

        # Only track the car with the selected target_id
        if track_id == target_id:
            cv2.rectangle(frame, (int(l), int(t)), (int(r), int(b)), (0, 255, 0), 3)
            cv2.putText(frame, f"Car ID: {track_id}", (int(l), int(t) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the tracking result
    cv2.imshow("Tracking One Car", frame)

    # Exit loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 2 cars, 4 trucks, 82.0ms
Speed: 2.0ms preprocess, 82.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 3 trucks, 75.0ms
Speed: 3.0ms preprocess, 75.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 3 trucks, 78.5ms
Speed: 4.0ms preprocess, 78.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Tracking car with ID: 1

0: 384x640 1 person, 3 cars, 3 trucks, 62.2ms
Speed: 2.0ms preprocess, 62.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 3 trucks, 1 clock, 74.5ms
Speed: 2.0ms preprocess, 74.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 3 trucks, 56.0ms
Speed: 2.0ms preprocess, 56.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 3 trucks, 57.5ms
Speed: 2.0ms preprocess, 57.5ms inference, 1.0ms postprocess per i

In [16]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('tracked_car.mp4', fourcc, 30, (int(cap.get(3)), int(cap.get(4))))